In [476]:
import pandas as pd
import re
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [477]:
%ls

Extended_dataset_researching.ipynb
Extending_dataset.ipynb
Genres.csv
Researching_dataset.ipynb
archive/
archive.zip
extended_dataset.csv
extended_dataset.csv.zip
kinopoisk-top250.csv
wiki_movie_plots_deduped.csv
Марат Гарафутдинов. Школа DS. Итоговый проект (1).docx
Кинопоиск.ipynb
Кинопоиск_250.ipynb


In [478]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')

In [479]:
df.head(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [480]:
df.info() # посмотрим на общую картинку

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


In [481]:
df[df['Genre'] == 'unknown'].count()

Release Year        6083
Title               6083
Origin/Ethnicity    6083
Director            6083
Cast                5400
Genre               6083
Wiki Page           6083
Plot                6083
dtype: int64

In [482]:
df.replace('unknown', np.nan, inplace=True) #Немного причешем датасет
df.replace('Unknown', np.nan, inplace=True)

In [483]:
df.dropna(subset=['Plot',], inplace=True) # выкинем все, что без описания и обновим индексы
df.reset_index(drop=True, inplace=True)

In [491]:
# df.drop('Unnamed: 0', axis=1, inplace=True) # дропнем лишнюю колонку

In [492]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,NaN,NaN,None,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,NaN,NaN,None,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,NaN,NaN,None,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,NaN,NaN,None,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,None,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


### Приведем жанры в порядок и построим по ним гистограмму

In [493]:
def replacing_film(x):
    if type(x) == str:
        return x.replace('film', '').lower().strip() # уберем слово "фильм", а также приведем всё к одному регистру.

In [494]:
df['Genre'] = df['Genre'].apply(replacing_film)

In [495]:
df['Genre'].nunique()

2158

In [496]:
most_freq_genres = df["Genre"].value_counts()
most_freq_genres[:50]

drama               5996
comedy              4403
horror              1173
action              1120
thriller             984
romance              940
western              865
science fiction      656
crime                568
adventure            528
musical              467
romantic comedy      465
crime drama          464
noir                 345
mystery              310
war                  278
animation            265
comedy, drama        240
family               231
animated             220
fantasy              204
musical comedy       154
comedy-drama         137
biography            136
anime                112
comedy drama         104
suspense             104
romantic drama       103
drama, romance        94
animated short        91
social                83
action thriller       79
historical            79
documentary           74
serial                71
world war ii          70
family drama          66
war drama             65
drama, crime          64
comedy, romance       63


In [497]:
df['Genre'] = df['Genre'].apply(lambda x: x.replace('sci-fi', 'science fiction').replace('science-fiction', 'science fiction') if x != None else None)

In [498]:
comedy_pattern = r'.*comedy.*'
drama_pattern = r'.*drama.*'
war_pattern = r'.*war.*'
thriller_pattern = r'.*thriller.*'

In [499]:
df['Genre'] = df['Genre'].apply(lambda x: x.replace(re.search(comedy_pattern, x).group(), 'comedy') 
                                if len(re.findall(comedy_pattern, str(x))) != 0 else x)
df['Genre'] = df['Genre'].apply(lambda x: x.replace(re.search(drama_pattern, x).group(), 'drama') 
                                if len(re.findall(drama_pattern, str(x))) != 0 else x)
df['Genre'] = df['Genre'].apply(lambda x: x.replace(re.search(war_pattern, x).group(), 'war') 
                                if len(re.findall(war_pattern, str(x))) != 0 else x)
df['Genre'] = df['Genre'].apply(lambda x: x.replace(re.search(thriller_pattern, x).group(), 'thriller') 
                                if len(re.findall(thriller_pattern, str(x))) != 0 else x)

In [500]:
most_freq_genres = df["Genre"].value_counts()
top25_genres = tuple(most_freq_genres[:25].index)
def regenrator(x): 
    '''Выцепим все самые популярные жанры из оставшихся записей'''
    if x is None:
        return np.nan
    if x in top25_genres:
        return x
    else:
        for i in top25_genres:
            if str(i) in str(x):
                return i
    return x

In [501]:
df["Genre"] = df['Genre'].apply(regenrator)

In [502]:
most_freq_genres = df["Genre"].value_counts()
top30_genres = tuple(most_freq_genres[:30].index)

In [503]:
top30_genres # их оставим, а остальное будет np.nan

('drama',
 'comedy',
 'action',
 'thriller',
 'horror',
 'romance',
 'western',
 'science fiction',
 'adventure',
 'crime',
 'musical',
 'war',
 'noir',
 'mystery',
 'animation',
 'family',
 'animated',
 'fantasy',
 'biography',
 'anime',
 'suspense',
 'historical',
 'social',
 'animated short',
 'documentary',
 'serial',
 'biopic',
 'superhero',
 'martial arts',
 'short')

In [504]:
def top_30_genres(x):
    'Оставим только 30 самых популярных жанров, остальное - nan'
    if x in top30_genres:
        return x
    else:
        return np.nan

In [505]:
df["Genre"] = df['Genre'].apply(top_30_genres)

In [506]:
most_freq_genres = df["Genre"].value_counts()
most_freq_genres

drama              8676
comedy             7266
action             1595
thriller           1571
horror             1321
romance            1044
western             895
science fiction     700
adventure           608
crime               598
musical             519
war                 426
noir                350
mystery             317
animation           293
family              287
animated            266
fantasy             239
biography           144
anime               117
suspense            104
historical           99
social               92
animated short       91
documentary          87
serial               71
biopic               57
superhero            49
martial arts         47
short                37
Name: Genre, dtype: int64

In [507]:
fig = px.histogram(x = most_freq_genres.values, y = most_freq_genres.index, height=600,
                  labels=dict(x='Количество фильмов', y='Жанр'),
                   title='Количество фильмов по 30 популярным жанрам'
                  )
fig.update_yaxes(categoryorder='total ascending')

In [508]:
# Наиболее популярные - драма и комедия. Разница с Экшеном в 4 раза. Ну и при этом данные жанры являются наиболее общими.

### Проанализируем режиссеров

In [509]:
df['Director'].nunique() # чересчур много. Посмотрим, как можно уменьшить.

12592

In [510]:
df['Director'] = df['Director'].apply(lambda x: x.replace('Director: ', '').replace('Directors: ','') if x is not np.nan else np.nan)
df['Director'] = df['Director'].apply(lambda x: x.strip() if x is not np.nan else np.nan)



In [511]:
df['Director'].nunique()

12262

In [512]:
df ['Director'] = df['Director'].apply(lambda x: str(x.split(';')[0].split(',')[0]) if x is not np.nan else np.nan)
# выберем только по первому режиссеру в списке, если их несколько

In [513]:
df['Director'].nunique()

11867

In [514]:
most_freq_directors = df["Director"].value_counts()
most_freq_directors[:50]

Michael Curtiz        80
Hanna-Barbera         77
Lloyd Bacon           70
Jules White           63
John Ford             61
Norman Taurog         58
Allan Dwan            58
Richard Thorpe        56
William A. Seiter     56
Mervyn LeRoy          55
Raoul Walsh           53
Alfred Hitchcock      53
William Beaudine      52
K. Raghavendra Rao    51
Gordon Douglas        51
K. Balachander        50
Alfred E. Green       50
Henry Hathaway        50
Roy Del Ruth          49
George Marshall       49
Robert Z. Leonard     48
Lew Landers           45
Ray Enright           45
William A. Wellman    45
Cecil B. DeMille      44
Lesley Selander       44
Michael Powell        43
Henry King            43
George Cukor          42
Woody Allen           42
Arthur Lubin          42
Maurice Elvey         40
Richard Fleischer     40
Edward Dmytryk        40
P. Vasu               40
J. Lee Thompson       39
George Sherman        39
Roger Corman          39
King Vidor            39
Val Guest             38


In [515]:
most_freq_directors[most_freq_directors == 1]

I.V.Sasi                  1
Ayush Raina               1
V. Ravindra               1
Sudipto Chattopadhyaya    1
Chitrayug                 1
                         ..
Jürgen Goslar             1
Horace Ové                1
John Sealey               1
Trevor Wrenn              1
Ferzan Özpetek            1
Name: Director, Length: 7292, dtype: int64

In [516]:
most_freq_directors.values.sum()

33762

In [517]:
most_freq_directors[:int(most_freq_directors.shape[0]*0.4)].values.sum()/most_freq_directors.values.sum()

0.7890824003317339

In [518]:
# Сделаем график зависимости количества фильмов от номера режиссера в списке, нарастающим итогом. С пересчетом на проценты.

In [519]:
x = []
y = []
for i in range(most_freq_directors.shape[0]):
    x.append(i/most_freq_directors.shape[0])
    y.append(most_freq_directors[:i+1].values.sum()/most_freq_directors.values.sum())

In [521]:
px.line(x=x,y=y,
       labels=dict(x = 'Процент от количества режиссеров', y = 'Процент от общего числа фильмов'),
       title='Зависимость количества фильмов от количества режиссеров')

In [522]:
# Хоть принцип Парето и не соблюдается, тем не менее. 80% фильмов было отснято чуть более чем 40% режиссерами.
# При этом около 60 процентов режиссеров в данном датасете отметились только одним фильмом. Видно по прямому графику.

In [523]:
fig = px.histogram(x=most_freq_directors.values[:50], y=most_freq_directors.index[:50], 
                  labels=dict(x='Количество отснятых фильмов', y='Режиссер'),
                  title='Количество отснятых фильмов по топ-50 режиссерам',
                  height=900
                  )
fig.update_yaxes(categoryorder='total ascending')

### Теперь сделаем гистограмму по нац принадлежности.

In [524]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,George S. Fleming,NaN,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [525]:
df ['Origin/Ethnicity'] = df['Origin/Ethnicity'].apply(lambda x: str(x.split(';')[0].split(',')[0]))
df['Origin/Ethnicity'].unique() #Вычистим непонятные значения.

array(['American', 'Australian', 'Bangladeshi', 'British', 'Canadian',
       'Chinese', 'Egyptian', 'Hong Kong', 'Filipino', 'Assamese',
       'Bengali', 'Bollywood', 'Kannada', 'Malayalam', 'Marathi',
       'Punjabi', 'Tamil', 'Telugu', 'Japanese', 'Malaysian', 'Maldivian',
       'Russian', 'South_Korean', 'Turkish'], dtype=object)

In [529]:
fig = px.histogram(data_frame=df[df['Origin/Ethnicity'] != None ],log_x=True, 
                  y = 'Origin/Ethnicity',
                  orientation='h',
                  height=600,
                  labels={'Origin/Ethnicity':'Нац. принадлежность'},  # Оперируя датафреймом, можно назвать только 1 ось
                  title="Количество отснятых фильмов по нац. принадлежности"
                  )
fig.update_yaxes(categoryorder='total ascending')

# Шкала логарифмическая из-за слишком больших разрывов между странами/языками.

In [530]:
# Больше всех снимают американцы. При этом разница со вторым местом - в 4 раза. * Только в рамках текущей выборки.

### Теперь проведем анализ по годам.

In [531]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,George S. Fleming,NaN,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [533]:
years_freq = df['Release Year'].value_counts()

In [555]:
years_freq.sort_index(inplace=True)

In [604]:
px.histogram(x=years_freq.index, y=years_freq.values, nbins=30, opacity=0.8,
            labels=dict(x='Год выпуска фильма', y='Количество отснятых фильмов'))

In [562]:
px.line(x=years_freq.index, y=years_freq.values, 
        labels=dict(x='Год выпуска фильма', y='Количество отснятых фильмов'), 
        title='График распределения количества выпущенных фильмов по годам'
       )

In [567]:
df1 = df[['Release Year', 'Origin/Ethnicity']]

In [572]:
years_origin_count = df1.value_counts()

In [579]:
years_origin_count.sort_index(inplace=True)

In [583]:
df1 = pd.DataFrame(years_origin_count).reset_index()

In [588]:
df1.columns = ['Release Year', 'Origin/Ethnicity', 'Num of films']

In [603]:
px.line(df1,x='Release Year', y='Num of films', color='Origin/Ethnicity',  
                title="Фильмы, отснятые в разных странах", log_y=False)

In [605]:
# В целом, общая тенденция идет на увеличение количества снимаемых фильмов. 
# При этом американцы снимают в разы больше фильмов, чем какие-либо другие страны.
# Также имеются периоды, в которые не идет резкого увеличения количества отснятых фильмов. 60-80 гг. 
# Вероятно, на то были экономические или политические причины.
# Если рассмотреть отдельно америку, то наблюдаются явные пики по выпуску фильмов с интервалом в несколько лет. 
# Возможно, это как-то связано с циклом производства.
